In [1]:
import json
import numpy as np
import threading
import time
import pandas as pd
from pcomp_utils.kafka_producer_utils import KafkaProducerHandler
from pcomp_utils.kafka_consumer_utils import KafkaConsumerHandler
from pcomp_utils.activation_functions import ACTIVATIONS, relu, softmax
from pcomp_utils.redis_utils import RedisHandler

# Kafka Configuration
KAFKA_BROKER = 'kafka:9092'


class Neuron(threading.Thread):
    def __init__(self, layer_id, neuron_id, weights, bias, activation, is_final_layer=False):
        threading.Thread.__init__(self)
        self.layer_id = layer_id
        self.neuron_id = neuron_id
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
        self.is_final_layer = is_final_layer
        self.output = None
        self.consumer = KafkaConsumerHandler(f'layer-{self.layer_id[-1]}', KAFKA_BROKER, partition=self.neuron_id)
        self.redis_handler = RedisHandler('host.docker.internal', 6379, 0)

    def fetch_input(self):
        return self.redis_handler.get("initial_data") if self.layer_id == 'layer_0' else self.redis_handler.get(f'layer_{int(self.layer_id[-1]) - 1}')

    def process_data(self, inputs):
        z = np.dot(inputs, self.weights) + self.bias
        return z if self.is_final_layer else self.activation_func(z)

    def run(self):
        for message in self.consumer.consume():
            if message.value.get('layer') == self.layer_id:
                input_data = self.fetch_input()
                self.output = self.process_data(input_data)
                print(f"✅ Neuron {self.neuron_id} in {self.layer_id} processed data.")
                break
        self.consumer.close()

class Layer:
    def __init__(self, layer_id, neuron_configs, is_final_layer=False):
        self.layer_id = layer_id
        self.neuron_configs = neuron_configs
        self.is_final_layer = is_final_layer
        self.neurons = []
        self.redis_handler = RedisHandler('host.docker.internal', 6379, 0)

    def initialize_neurons(self):
        self.neurons = [
            Neuron(
                layer_id=self.layer_id,
                neuron_id=idx,
                weights=neuron['weights'],
                bias=neuron['biases'],
                activation=neuron['activation'],
                is_final_layer=self.is_final_layer
            )
            for idx, neuron in enumerate(self.neuron_configs)
        ]

    def forward(self, image_id):
        self.initialize_neurons()

        # Start neuron threads
        for neuron in self.neurons:
            neuron.start()

        time.sleep(2)

        # Send activation message to each neuron's partition
        producer = KafkaProducerHandler(KAFKA_BROKER)

        for neuron_id in range(len(self.neurons)):
            activation_message = {'layer': self.layer_id}
            producer.send(f'layer-{self.layer_id[-1]}', activation_message, neuron_id)
            #print(f"✅ Layer {self.layer_id} sent activation to Neuron {neuron_id} on partition {neuron_id}")

        producer.close()

        # Wait for all neuron threads to complete
        for neuron in self.neurons:
            neuron.join()

        # Aggregate and store neuron outputs
        outputs = np.array([neuron.output for neuron in self.neurons])
        print("Outputs:")
        print(outputs)
        self.redis_handler.set(self.layer_id, outputs)
        print(f"📝 Layer {self.layer_id} stored aggregated data in Redis.")

        if self.is_final_layer:
            prediction = int(np.argmax(outputs))
            self.redis_handler.hset('predictions', image_id, prediction)
            print(f"🎯 Prediction for Image {image_id}: {prediction}")

        if not self.is_final_layer:
            self.activate_next_layer()

    def activate_next_layer(self):
        producer = KafkaProducerHandler(KAFKA_BROKER)
        next_layer = f'layer_{int(self.layer_id[-1]) + 1}'
        print(f"🚀 Activating next layer: {next_layer}")
        producer.send('activate-layer', {'layer': next_layer})
        producer.close()



def store_initial_input_data(input_data):
    redis_handler = RedisHandler('host.docker.internal', 6379, 0)
    redis_handler.set("initial_data", input_data)
    print("📥 Initial input data stored in Redis under 'initial_data' key.")

def calculate_accuracy(file_path, limit=10):
    redis_handler = RedisHandler('host.docker.internal', 6379, 0)
    df = pd.read_csv(file_path).head(limit)
    predictions = redis_handler.hgetall('predictions')

    correct = 0
    total = 0

    for idx, row in df.iterrows():
        label = int(row['label'])
        prediction = predictions.get(str(idx).encode())  # Decode Redis key lookup

        if prediction is not None:
            prediction = int(prediction.decode())  # Decode the Redis stored value
            print(f"✅ Prediction for Image {idx}: {prediction}, Actual Label: {label}")

            if prediction == label:
                correct += 1
            total += 1
        else:
            print(f"⚠️ No prediction found for Image ID: {idx}")

    if total == 0:
        print("⚠️ No valid predictions to calculate accuracy.")
        return

    accuracy = correct / total
    print(f"🎯 Test Accuracy (First {limit} Images): {accuracy * 100:.2f}%")

def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

def build_network(json_data):
    layers = []
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))
    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        neuron_configs = layer_info['nodes']
        layers.append(Layer(layer_id=layer_name, neuron_configs=neuron_configs, is_final_layer=(i == len(sorted_layers) - 1)))
    return layers

def forward_pass(layers, image_np, image_id):
    store_initial_input_data(image_np)
    for layer in layers:
        layer.forward(image_id)

# Load network
data = load_network("node_based_model.json")
network = build_network(data)


df = pd.read_csv('data/mnist.csv').head(2)  # Only the first 10 images

for idx, row in df.iterrows():
    image_np = row.iloc[:-1].values.astype(np.float64)  # Extract image pixels
    label = row.iloc[-1]  # Extract label
    forward_pass(network, image_np, idx)

# Calculate and print accuracy
calculate_accuracy('data/mnist.csv', 2)

📥 Initial input data stored in Redis under 'initial_data' key.


KeyboardInterrupt: 